# Messaging and Requests

## Install

In [1]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html

In [2]:
import syft as sy
sy.requires(SYFT_VERSION)

✅ The installed version of syft==0.8.2b2 matches the requirement >=0.8.2b0 and the requirement <0.9


In [3]:
node = sy.orchestra.launch(name="messages-requests-example-domain-1-ds",port=7023, reset=True)

Starting messages-requests-example-domain-1-ds server on 0.0.0.0:7023


Waiting for server to start Done.


## Setup

For the purpose of this tutorial we are creating a very simple dataset, which is created and owner by the root client

In [4]:
admin_client = node.login(email="info@openmined.org", password="changethis")

Logged into messages-requests-example-domain-1-ds as <info@openmined.org>


In [5]:
import numpy as np
dataset = sy.Dataset(
    name="my dataset",
    asset_list=[
        sy.Asset(
        name="my asset",
        data=np.array([1,2,3]),
        mock=np.array([1,1,1])
    )]
)
admin_client.upload_dataset(dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 14.95it/s]

Uploading: my asset


<class 'syft.service.response.SyftSuccess'>: Dataset uploaded to 'messages-requests-example-domain-1-ds'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [6]:
admin_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")

<class 'syft.service.response.SyftSuccess'>: User 'Jane Doe' successfully registered! To see users, run `[your_client].users`

In [7]:
guest_client = node.client.login(email="jane@caltech.edu", password="abc123")

Logged into messages-requests-example-domain-1-ds as <jane@caltech.edu>


In [8]:
asset = guest_client.datasets[0].assets[0]

In [9]:
@sy.syft_function_single_use()
def func():
    import random
    data = list(range(100))
    return sum(data) / 100 + random.random()

@sy.syft_function_single_use(data=asset)
def mean(data):
    import random
    return sum(data)/len(data) + + random.random()

@sy.syft_function_single_use(data=asset)
def reveal_data(data):
    return data

Syft function 'func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
Syft function 'mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
Syft function 'reveal_data' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.


In [10]:
# Create a project
new_project = sy.Project(
    name="My Cool Project",
    description="""Hi, I want to calculate the mean of your private data,\
                    pretty please!""",
    members=[guest_client],
)
new_project

syft.service.project.project.ProjectSubmit

In [11]:
new_project.create_code_request(func, guest_client)
new_project.create_code_request(mean, guest_client)
new_project.create_code_request(reveal_data, guest_client)

<class 'syft.service.response.SyftSuccess'>: Code request for 'reveal_data' successfully added to 'My Cool Project' Project. To see code requests by a client, run `[your_client].code`

In [12]:
project = new_project.start()
project

syft.service.project.project.Project

In [13]:
dangerous_request = admin_client.projects[0].requests[-1]
dangerous_request.deny(reason="Dangerous request, you just want to reveal the data!")

<class 'syft.service.response.SyftSuccess'>: Request 9e099695e5a04d6daf6a0ab42eab9ff3 successfully denied !

## Messaging

list notifications using client.notifications (messages sent and requests)


In [14]:
guest_client.notifications

If possible try using client.requests/client.projects


## Common Permission Errors

approve request that you dont have permission for example

In [15]:
guest_client.projects

In [16]:
guest_client.projects[0]

syft.service.project.project.Project

In [17]:
func_request = guest_client.projects[0].requests[0]
func_request

syft.service.request.request.Request

In [18]:
func_request.approve()

---------------------------------------------------------------------------
SyftAttributeError
---------------------------------------------------------------------------
Exception: 'APIModule' api.request object has no submodule or method 'apply', you may not have permission to access the module you are trying to access


## Requesting Changes

request permission to an object via a pointer

In [ ]:
asset = guest_client.datasets[0].assets[0]
asset

In [ ]:
request = asset.pointer.request(guest_client)
request